<a href="https://colab.research.google.com/github/Kwesisbits/DataScience/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [107]:
credit_card_data = pd.read_csv('/content/credit_data.csv')

In [108]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [109]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [110]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [111]:
# distribution of legit and fradulent transactions
credit_card_data['Class'].value_counts()

,count
Class,
0,284315
1,492


Datset is extremely unbalanced

0 -> Legit transaction

1 -> Fraudulent transaction

In [112]:
# Separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [113]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [114]:
#statistical measures of the data
legit.Amount.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [115]:
fraud.Amount.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [116]:
#compare values for both classes
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under-Sampling

Building sample dataset with similar distribution of legit and fraudulent transactions

In [117]:
legit_sample = legit.sample(n=492)

Concatenating two DataFrames

In [118]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
181481,124995.0,1.373059,-2.254168,-0.998745,-0.607998,-1.271643,0.083727,-0.564922,0.067321,-0.034650,0.646401,0.755539,-0.009410,-0.757156,-0.075200,-0.937751,1.162171,0.232738,-1.246412,1.015654,0.705951,0.063390,-0.714418,0.132931,0.664332,-0.679289,-0.660826,-0.074299,0.018337,393.11,0
270322,164021.0,-1.872971,2.274707,-2.307026,-1.270305,0.140916,-1.290272,0.523513,0.900279,-0.460571,-0.031089,0.003419,0.607487,-0.754817,1.727648,-0.794123,-0.087233,-0.237611,0.288982,0.171335,-0.245072,0.380184,0.858695,-0.094814,-0.439771,-0.182015,0.079037,-0.171442,0.178199,19.99,0
58005,48186.0,1.231254,0.283608,0.185485,0.502945,-0.159803,-0.560402,-0.022205,-0.034239,-0.199255,-0.113473,1.332462,0.795181,0.161226,-0.102435,0.422010,0.772839,-0.373126,0.308522,0.214252,-0.055964,-0.256290,-0.782455,0.074397,-0.037832,0.227524,0.097152,-0.028081,0.017705,4.49,0
31843,36538.0,0.713301,-0.707770,1.027689,1.868472,-0.823121,0.687566,-0.227401,0.189979,1.108064,-0.528660,-0.968228,1.414820,0.565529,-0.858472,-1.767437,-1.159046,0.734442,-1.412907,0.108387,0.229115,-0.291236,-0.754338,-0.092442,0.113187,0.383095,-0.532661,0.060630,0.063555,200.63,0
24025,33032.0,-0.681865,1.078995,1.899994,2.593872,0.298132,0.454279,0.329257,0.131723,-0.830908,0.556694,-0.734007,-0.041935,-0.019311,-0.184748,-0.079206,-0.532609,0.380225,-0.851484,0.183262,-0.053384,-0.105598,-0.236537,0.060935,0.065115,-0.619287,-0.121746,-0.108341,0.218682,5.71,0


In [119]:
new_dataset['Class'].value_counts()

,count
Class,
0,492
1,492


In [120]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,93280.211382,-0.033675,-0.006054,-0.042212,-0.010930,-0.068321,0.038558,0.000851,0.049339,-0.026005,-0.059464,-0.011859,-0.053972,0.010952,0.027427,-0.011094,0.058548,-0.010694,-0.019923,0.018005,0.027322,-0.012207,-0.038244,-0.023061,0.051905,-0.003406,-0.016151,-0.033127,0.022175,105.705955
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Splitting the dataset

In [121]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [122]:
print(X)

            Time        V1        V2  ...       V27       V28  Amount
181481  124995.0  1.373059 -2.254168  ... -0.074299  0.018337  393.11
270322  164021.0 -1.872971  2.274707  ... -0.171442  0.178199   19.99
58005    48186.0  1.231254  0.283608  ... -0.028081  0.017705    4.49
31843    36538.0  0.713301 -0.707770  ...  0.060630  0.063555  200.63
24025    33032.0 -0.681865  1.078995  ... -0.108341  0.218682    5.71
...          ...       ...       ...  ...       ...       ...     ...
279863  169142.0 -1.927883  1.125653  ...  0.292680  0.147968  390.00
280143  169347.0  1.378559  1.289381  ...  0.389152  0.186637    0.76
280149  169351.0 -0.676143  1.126366  ...  0.385107  0.194361   77.89
281144  169966.0 -3.113832  0.585864  ...  0.884876 -0.253700  245.00
281674  170348.0  1.991976  0.158476  ...  0.002988 -0.015309   42.53

[984 rows x 30 columns]


In [123]:
print(Y)

181481    0
270322    0
58005     0
31843     0
24025     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [124]:
#splitting data into test and training data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [125]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


Model Training with Logistic Regression

In [126]:
model = LogisticRegression(max_iter=10000)

In [127]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=10000)

Model Evaluation

In [128]:
#accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy on training data :', training_data_accuracy )

Accuracy on training data : 0.963151207115629


In [131]:
#accuracy on testing data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy on testing data :', test_data_accuracy )

Accuracy on testing data : 0.9187817258883249


In [134]:
#Confusion matrix
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
print(confusion_matrix(Y_test, X_test_prediction))
print(classification_report(Y_test, X_test_prediction))


[[93  6]
 [10 88]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92        99
           1       0.94      0.90      0.92        98

    accuracy                           0.92       197
   macro avg       0.92      0.92      0.92       197
weighted avg       0.92      0.92      0.92       197

